This notebook is designed to address the question why recall in iteration 3 is smaller than previous iterations. What happens to the S wave not detected by the new iteration.

In [1]:
import pandas as pd

In [2]:
reference_phase=pd.read_csv("/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/tonga_picks_updated_2023_0426.csv")
reference_phase_melted = reference_phase.melt(id_vars=['originid', 'stacode'], 
                    value_vars=['arrival_time_P', 'arrival_time_S'], 
                    var_name='type', 
                    value_name='time')
reference_phase_melted['type'] = reference_phase_melted['type'].str.replace('arrival_time_', '')
reference_phase_melted["type"]=reference_phase_melted["type"].replace({"P":"p", "S":"s"})
reference_phase_melted["time"]=pd.to_datetime(reference_phase_melted["time"])
reference_phase_melted.rename(columns={"stacode":"sta", "originid":"mp_event_index"}, inplace=True)
reference_phase_melted.dropna(inplace=True)
reference_phase=reference_phase_melted
reference_phase.head(),len(reference_phase)

(  mp_event_index   sta type                       time
 0       11_52111   A01    p 2009-12-02 01:14:41.317870
 1       11_52111  A02W    p 2009-12-02 01:14:42.457750
 2       11_52111   A03    p 2009-12-02 01:14:43.547000
 3       11_52111   A05    p 2009-12-02 01:14:47.038000
 4       11_52111  A06W    p 2009-12-02 01:14:48.857230,
 57108)

In [3]:
len(reference_phase[reference_phase.type=="p"]),len(reference_phase[reference_phase.type=="s"])

(42256, 14852)

In [4]:
reference_catalog=pd.read_csv("/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/tonga_catalog_updated_2023_0426.csv")
reference_catalog.drop(columns=["eventid","author","subset","mb","ISCid"],inplace=True)
reference_catalog["time"]=pd.to_datetime(reference_catalog["time"])
reference_catalog.rename(columns={"originid":"mp_event_index"},inplace=True)
reference_catalog.head(),len(reference_catalog)

(   latitude  longitude     depth                       time mp_event_index
 0  -21.4486  -175.0123   59.8167 2009-12-02 01:14:28.278360       11_52111
 1  -21.0021  -176.0597  189.5062 2009-12-02 02:25:13.607840       11_52113
 2  -20.0291  -175.3772  179.2744 2009-12-02 15:41:32.776970       11_52115
 3  -20.8052  -174.5321   66.1092 2009-12-03 09:43:16.236000       11_52119
 4  -19.4698  -174.9927  146.2841 2009-12-04 17:37:18.418720       11_52124,
 1163)

In [5]:
iteration1_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continious_bootstrapped.csv",
        usecols=[0, 1, 2, 3, 4],
    )
iteration1_catalog.rename(
        columns={
            "id": "event_index",
        },
        inplace=True,
    )
iteration1_catalog["time"] = pd.to_datetime(iteration1_catalog["datetime"])
iteration1_catalog.drop(columns=["datetime"], inplace=True)
iteration1_catalog.head(),len(iteration1_catalog)

(   event_index   latitude   longitude       depth  \
 0           55 -21.078356 -177.229740  365.655880   
 1           68 -16.359084 -174.505760  204.380435   
 2           74 -20.494930 -176.895955  478.382117   
 3           88 -17.442211 -177.878019  646.372351   
 4           93 -18.800355 -175.508731   28.436913   
 
                            time  
 0 2010-01-18 06:48:53.679099136  
 1 2010-01-26 01:47:51.560340480  
 2 2010-06-28 04:23:34.643994112  
 3 2010-04-16 04:21:24.130119936  
 4 2010-07-12 04:08:03.839479552  ,
 9427)

In [6]:
iteration1_phasenettf_phase=pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continious_associated_assignment.csv",
        skiprows=1,
        names=[
            "id",
            "date",
            "time",
            "amp",
            "type",
            "prob",
            "event_index",
            "gamma_score",
        ],
        sep=r"\s+",
    )

iteration1_phasenettf_phase["sta"]=iteration1_phasenettf_phase["id"].str.split(".").str[1]
iteration1_phasenettf_phase["time"]=pd.to_datetime(iteration1_phasenettf_phase["date"]+" "+iteration1_phasenettf_phase["time"])
iteration1_phasenettf_phase=iteration1_phasenettf_phase.drop(["date","id","amp","prob"],axis=1)
iteration1_phasenettf_phase.head(),len(iteration1_phasenettf_phase)

(                        time type  event_index  gamma_score   sta
 0 2009-11-01 12:29:38.419900    p           -1    -1.000000  MSVF
 1 2009-11-01 16:03:19.149900    p        31625     0.079182  MSVF
 2 2009-11-01 20:23:03.249900    p        12394     0.068525  MSVF
 3 2009-11-01 20:59:34.149900    p           -1    -1.000000  MSVF
 4 2009-11-01 20:23:15.619900    s        12394     0.052664  MSVF,
 749549)

In [7]:
# select iteration1_relocation_phase from iteration1_phasenettf_phase with event_index in iteration1_relocation_catalog
iteration1_relocation_catalog_event_index_set=set(iteration1_catalog["event_index"])
iteration1_relocation_phase=iteration1_phasenettf_phase[iteration1_phasenettf_phase["event_index"].isin(iteration1_relocation_catalog_event_index_set)]
iteration1_relocation_phase.head(),len(iteration1_relocation_phase)

(                         time type  event_index  gamma_score   sta
 21 2009-11-04 20:43:54.029900    p        15715     0.083852  MSVF
 70 2009-11-10 08:05:06.929900    p         6004     0.088009  MSVF
 72 2009-11-10 08:17:57.669900    p        21695     0.113771  MSVF
 76 2009-11-10 08:05:30.519900    s         6004     0.002691  MSVF
 77 2009-11-10 08:18:55.549900    s        21695     0.108274  MSVF,
 280844)

In [8]:
len(iteration1_relocation_phase[iteration1_relocation_phase.type=="p"]),len(iteration1_relocation_phase[iteration1_relocation_phase.type=="s"])

(217254, 63590)

In [9]:
iteration2_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continious_semi.csv",
        usecols=[0, 1, 2, 3, 4],
    )
iteration2_catalog.rename(
        columns={
            "id": "event_index",
        },
        inplace=True,
    )
iteration2_catalog["time"] = pd.to_datetime(iteration2_catalog["datetime"])
iteration2_catalog.drop(columns=["datetime"], inplace=True)
iteration2_catalog.head(),len(iteration2_catalog)

(   event_index   latitude   longitude       depth  \
 0           49 -24.055055 -178.753175  581.541092   
 1           59 -22.996563 -175.655182   85.202101   
 2           64 -23.101836 -178.642513  554.767645   
 3           78 -21.863043 -175.590572  140.706452   
 4           95 -20.365366 -177.123331  446.403303   
 
                            time  
 0 2010-09-05 06:20:43.693259008  
 1 2010-02-03 07:50:12.698582784  
 2 2010-07-21 13:36:11.897348096  
 3 2010-02-07 15:23:13.122398720  
 4 2010-04-16 07:59:40.670819840  ,
 13799)

In [10]:
iteration2_phasenettf_phase=pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/semi-v1.assignment.csv",
        skiprows=1,
        names=[
            "id",
            "date",
            "time",
            "amp",
            "type",
            "prob",
            "event_index",
            "gamma_score",
        ],
        sep=r"\s+",
    )

iteration2_phasenettf_phase["sta"]=iteration2_phasenettf_phase["id"].str.split(".").str[1]
iteration2_phasenettf_phase["time"]=pd.to_datetime(iteration2_phasenettf_phase["date"]+" "+iteration2_phasenettf_phase["time"])
iteration2_phasenettf_phase=iteration2_phasenettf_phase.drop(["date","id","amp","prob"],axis=1)
iteration2_phasenettf_phase.head(),len(iteration2_phasenettf_phase)

(                        time type  event_index  gamma_score   sta
 0 2009-11-01 04:21:03.099900    p        25214     0.002465  MSVF
 1 2009-11-01 08:07:02.399900    p           -1    -1.000000  MSVF
 2 2009-11-01 12:29:38.529900    p           -1    -1.000000  MSVF
 3 2009-11-01 12:57:23.319900    p           -1    -1.000000  MSVF
 4 2009-11-01 16:03:19.029900    p        12931     0.077535  MSVF,
 1293246)

In [11]:
iteration2_relocation_catalog_event_index_set=set(iteration2_catalog["event_index"])
iteration2_relocation_phase=iteration2_phasenettf_phase[iteration2_phasenettf_phase["event_index"].isin(iteration2_relocation_catalog_event_index_set)]
iteration2_relocation_phase.head(),len(iteration2_relocation_phase)

(                          time type  event_index  gamma_score   sta
 62  2009-11-04 20:43:53.969900    p        98879     0.005614  MSVF
 81  2009-11-06 16:32:12.919900    p        79255     0.071732  MSVF
 112 2009-11-08 16:39:36.679900    p        98419     0.111880  MSVF
 163 2009-11-10 08:05:07.049900    p        71320     0.095103  MSVF
 165 2009-11-10 08:17:57.599900    p        14608     0.114541  MSVF,
 422840)

In [12]:
len(iteration2_relocation_phase[iteration2_relocation_phase.type=="p"]),len(iteration2_relocation_phase[iteration2_relocation_phase.type=="s"])

(343247, 79593)

In [13]:
iteration3_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continuous_v3.csv",
        usecols=[0, 1, 2, 3, 4],
    )
iteration3_catalog.rename(
        columns={
            "id": "event_index",
        },
        inplace=True,
    )
iteration3_catalog["time"] = pd.to_datetime(iteration3_catalog["datetime"])
iteration3_catalog.drop(columns=["datetime"], inplace=True)
iteration3_catalog.head(),len(iteration3_catalog)

(   event_index   latitude   longitude       depth  \
 0          188 -16.201434 -172.365634  106.331183   
 1          210 -18.773165 -175.619987  214.956622   
 2          221 -20.299335 -174.154085  129.158232   
 3          251 -16.156015 -172.249285   93.169231   
 4          258 -18.484624 -174.527434  209.984489   
 
                            time  
 0 2009-11-20 01:53:42.226608896  
 1 2009-11-19 18:36:05.594294272  
 2 2009-11-30 08:20:48.752517632  
 3 2009-11-23 21:30:12.039857920  
 4 2009-11-24 05:30:48.569999872  ,
 13406)

In [14]:
iteration3_phasenettf_phase=pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/semi-iteration3_picks.csv",
        skiprows=1,
        names=[
            "id",
            "date",
            "time",
            "amp",
            "type",
            "prob",
            "event_index",
            "gamma_score",
        ],
        sep=r"\s+",
    )

iteration3_phasenettf_phase["sta"]=iteration3_phasenettf_phase["id"].str.split(".").str[1]
iteration3_phasenettf_phase["time"]=pd.to_datetime(iteration3_phasenettf_phase["date"]+" "+iteration3_phasenettf_phase["time"])
iteration3_phasenettf_phase=iteration3_phasenettf_phase.drop(["date","id","amp","prob"],axis=1)
iteration3_phasenettf_phase.head(),len(iteration3_phasenettf_phase)

(                        time type  event_index  gamma_score   sta
 0 2009-11-01 04:21:04.899900    p         1347     0.001252  MSVF
 1 2009-11-30 05:36:21.775600    p         3667     0.000773  C09W
 2 2009-11-30 05:44:57.775600    p         3670     0.000773  C09W
 3 2009-11-30 05:36:45.875600    s         3667     0.000142  C09W
 4 2009-11-30 09:03:03.445600    p         5329     0.000141  C09W,
 2080940)

In [15]:
iteration3_relocation_catalog_event_index_set=set(iteration3_catalog["event_index"])
iteration3_relocation_phase=iteration3_phasenettf_phase[iteration3_phasenettf_phase["event_index"].isin(iteration3_relocation_catalog_event_index_set)]
iteration3_relocation_phase.head(),len(iteration3_relocation_phase)

(                         time type  event_index  gamma_score   sta
 1  2009-11-30 05:36:21.775600    p         3667     0.000773  C09W
 3  2009-11-30 05:36:45.875600    s         3667     0.000142  C09W
 10 2009-11-30 13:22:19.465600    p         2816     0.000107  C09W
 14 2009-11-30 13:22:44.345600    s         2816     0.000412  C09W
 18 2009-11-30 01:14:11.345600    s         1746     0.093520  C09W,
 451627)

In [16]:
len(iteration3_relocation_phase[iteration3_relocation_phase.type=="p"]),len(iteration3_relocation_phase[iteration3_relocation_phase.type=="s"])

(372774, 78853)

# Analysis

In [17]:
reference_phase.head()

,mp_event_index,sta,type,time
0,11_52111,A01,p,2009-12-02 01:14:41.317870
1,11_52111,A02W,p,2009-12-02 01:14:42.457750
2,11_52111,A03,p,2009-12-02 01:14:43.547000
3,11_52111,A05,p,2009-12-02 01:14:47.038000
4,11_52111,A06W,p,2009-12-02 01:14:48.857230


In [18]:
iteration1_relocation_phase.head()

,time,type,event_index,gamma_score,sta
21,2009-11-04 20:43:54.029900,p,15715,0.083852,MSVF
70,2009-11-10 08:05:06.929900,p,6004,0.088009,MSVF
72,2009-11-10 08:17:57.669900,p,21695,0.113771,MSVF
76,2009-11-10 08:05:30.519900,s,6004,0.002691,MSVF
77,2009-11-10 08:18:55.549900,s,21695,0.108274,MSVF


In [19]:
iteration3_relocation_phase.head()

,time,type,event_index,gamma_score,sta
1,2009-11-30 05:36:21.775600,p,3667,0.000773,C09W
3,2009-11-30 05:36:45.875600,s,3667,0.000142,C09W
10,2009-11-30 13:22:19.465600,p,2816,0.000107,C09W
14,2009-11-30 13:22:44.345600,s,2816,0.000412,C09W
18,2009-11-30 01:14:11.345600,s,1746,0.093520,C09W


In [30]:
ref=reference_phase[reference_phase.type=="s"]
iter1=iteration1_relocation_phase[iteration1_relocation_phase.type=="s"]
iter3=iteration3_relocation_phase[iteration3_relocation_phase.type=="s"]

ref=ref.sort_values(by=["time","sta"])
iter1=iter1.sort_values(by=["time","sta"])
iter3=iter3.sort_values(by=["time","sta"])

merged1=pd.merge_asof(ref,iter1,on="time",by="sta",direction="nearest",tolerance=pd.Timedelta("1s"))
merged3=pd.merge_asof(ref,iter3,on="time",by="sta",direction="nearest",tolerance=pd.Timedelta("1s"))

# select merged1 so type_y is not null
merged1=merged1[merged1["type_y"].notnull()]
# select merged3 so type_y is null
merged3=merged3[merged3["type_y"].isnull()]

# see which rows in merged1 and merged3 have the same mp_event_index,sta
common=merged1.merge(merged3,on=["mp_event_index","sta"],how="inner")
common.head(n=20)

,mp_event_index,sta,type_x_x,time_x,type_y_x,event_index_x,gamma_score_x,type_x_y,time_y,type_y_y,event_index_y,gamma_score_y
0,1_51844,B03,s,2009-12-01 04:08:48.357460000,s,3902.0,0.000375,s,2009-12-01 04:08:48.357460000,NaN,NaN,NaN
1,1_51864,A14W,s,2009-12-01 23:24:01.470030000,s,34487.0,0.027063,s,2009-12-01 23:24:01.470030000,NaN,NaN,NaN
2,1_51866,F02W,s,2009-12-02 06:32:40.700790000,s,39624.0,0.096536,s,2009-12-02 06:32:40.700790000,NaN,NaN,NaN
3,1_51870,VABL,s,2009-12-02 13:56:56.876320000,s,37929.0,0.036145,s,2009-12-02 13:56:56.876320000,NaN,NaN,NaN
4,1_51874,CICA,s,2009-12-02 21:03:51.586410000,s,46082.0,0.131772,s,2009-12-02 21:03:51.586410000,NaN,NaN,NaN
5,1_51874,B09W,s,2009-12-02 21:04:04.059550000,s,46082.0,0.131727,s,2009-12-02 21:04:04.059550000,NaN,NaN,NaN
6,1_51874,A14W,s,2009-12-02 21:04:06.236030000,s,46082.0,0.129078,s,2009-12-02 21:04:06.236030000,NaN,NaN,NaN
7,1_51874,C10W,s,2009-12-02 21:04:12.720700000,s,46082.0,0.140003,s,2009-12-02 21:04:12.720700000,NaN,NaN,NaN
8,1_51874,B02,s,2009-12-02 21:04:13.044220000,s,46082.0,0.086811,s,2009-12-02 21:04:13.044220000,NaN,NaN,NaN
9,1_51874,C03W,s,2009-12-02 21:04:17.063630000,s,46082.0,0.056349,s,2009-12-02 21:04:17.063630000,NaN,NaN,NaN
